# Comparison using kerchunk / datatree

In [2]:
from datatree import DataTree
from datatree import map_over_subtree
import xarray as xr
import pandas as pd
import dask
import xclim
import thermofeel as tf
import numpy as np 
from distributed import Client
from fsspec.implementations.reference import ReferenceFileSystem

from utils import wbgt, load_elev, adjust_pressure

In [8]:
# Read the reference catalog into a Pandas DataFrame
cat_df = pd.read_csv(
    "s3://carbonplan-share/nasa-nex-reference/reference_catalog_nested.csv"
)
cat_df = cat_df.iloc[0:2]
# Convert the DataFrame into a dictionary
catalog = cat_df.set_index("ID").T.to_dict("records")[0]

In [9]:
def load_ref_ds(url: str, gcm_scenario: str):
    storage_options={"remote_protocol":"s3","target_protocol":"s3", "lazy":True, "skip_instance_cache": True} # options passed to fsspec
    open_dataset_options={"chunks": {}} # opens passed to xarray
    ds = xr.open_dataset(url, engine="kerchunk", storage_options=storage_options, open_dataset_options=open_dataset_options)
    
    # adding the gcm/scenario combo to attrs for later down the pipeline
    ds.attrs['gcm_scenario'] = gcm_scenario
    return ds 


tasks = {id: dask.delayed(load_ref_ds)(url, id) for id, url in catalog.items()}

In [4]:
client = Client(n_workers=8)
client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49209,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:49236,Total threads: 1
Dashboard: http://127.0.0.1:49240/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:49212,


In [6]:
catalog_computed = dask.compute(tasks)

In [7]:
dt = DataTree.from_dict(catalog_computed[0])
elev = load_elev()


In [8]:

def calc_wbgt(ds):
    ds = ds.to_dataset()
    ds = ds.isel(time=slice(0,365))
    # calculate elevation-adjusted pressure
    ds["ps"] = xr.apply_ufunc(
        adjust_pressure, ds["tas"], elev, dask="allowed"
    ).rename({"elevation": "ps"})["ps"]
    ds["ps"].attrs["units"] = "Pa"
    ds["hurs"] = xclim.indices.relative_humidity(
        tas=ds["tasmax"], huss=ds["huss"], ps=ds["ps"]
    )
    ds["tasmax"].attrs = {}

    # windspeed assumption of 0.5 m/s (approximating shaded/indoor
    # conditions)
    ds["sfcWind"] = (ds["tas"] - ds["tas"]) + 0.5
    ds["WBT"] = tf.thermofeel.calculate_wbt(
        ds["tasmax"] - 273.15, ds["hurs"]
    )

    ds["BGT"] = tf.thermofeel.calculate_bgt(
        ds["tasmax"], ds["tasmax"], ds["sfcWind"]
    )
    ds["WBGT"] = wbgt(ds["WBT"], ds["BGT"], ds["tasmax"] - 273.15)
    ds["WBGT"].attrs["units"] = "degC"
    ds = ds[["WBGT"]]
    ds = dask.optimize(ds)[0]
    output = (
        f"s3://carbonplan-scratch/TEMP_NASA_NEX/wbgt-shade-"
        f"gridded/years/{ds.attrs['gcm_scenario']}.zarr"
    )

    
    return ds.to_zarr(output, consolidated=True, compute=False, mode="w")
    

delay_ds_list = [calc_wbgt(ds) for ds in dt.leaves]


In [9]:
dask.compute(delay_ds_list)

([None, None],)

In [ ]:


# @map_over_subtree
# def calc_wbgt(ds):
#     ds = ds.isel(time=slice(0,1))
#     # calculate elevation-adjusted pressure
#     ds["ps"] = xr.apply_ufunc(
#         adjust_pressure, ds["tas"], elev, dask="allowed"
#     ).rename({"elevation": "ps"})["ps"]
#     ds["ps"].attrs["units"] = "Pa"
#     ds["hurs"] = xclim.indices.relative_humidity(
#         tas=ds["tasmax"], huss=ds["huss"], ps=ds["ps"]
#     )
#     ds["tasmax"].attrs = {}

#     # windspeed assumption of 0.5 m/s (approximating shaded/indoor
#     # conditions)
#     ds["sfcWind"] = (ds["tas"] - ds["tas"]) + 0.5
#     ds["WBT"] = tf.thermofeel.calculate_wbt(
#         ds["tasmax"] - 273.15, ds["hurs"]
#     )

#     ds["BGT"] = tf.thermofeel.calculate_bgt(
#         ds["tasmax"], ds["tasmax"], ds["sfcWind"]
#     )
#     ds["WBGT"] = wbgt(ds["WBT"], ds["BGT"], ds["tasmax"] - 273.15)
#     ds["WBGT"].attrs["units"] = "degC"
#     ds = ds[["WBGT"]]
#     ds = dask.optimize(ds)[0]
#     output = (
#         f"s3://carbonplan-scratch/TEMP_NASA_NEX/wbgt-shade-"
#         f"gridded/years/{ds.attrs['gcm_scenario']}.zarr"
#     )

    
#     # ds.to_zarr(output, consolidated=True, mode="w")
#     return ds

# delayed_datatree = calc_wbgt(dt)


In [ ]:
output = ("s3://carbonplan-scratch/TEMP_NASA_NEX/DATATREE_TEST.zarr")
delayed_to_zarr = delayed_datatree.to_zarr(output, compute=False)



In [ ]:
delayed_obs.compute()

In [ ]:
38 secs for single time slice and 10 gcms